In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install optuna
!pip install tabpfn
!pip install catboost
!pip install confidenceinterval
!pip install shap

In [ ]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from matplotlib.font_manager import FontProperties
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso
from sklearn.decomposition import PCA
from sklearn.metrics import roc_curve, precision_recall_curve, precision_score, recall_score, f1_score, accuracy_score, matthews_corrcoef, roc_auc_score, average_precision_score, brier_score_loss, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

from imblearn.over_sampling import SMOTE

from tabpfn import TabPFNClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier

from confidenceinterval.bootstrap import bootstrap_ci

import optuna
from optuna.samplers import TPESampler

import shap

import warnings
warnings.filterwarnings('ignore')

In [ ]:
!wget https://github.com/adachis/Lifehacker.me/raw/master/fonts/HelveticaNeue.ttf
!wget https://github.com/adachis/Lifehacker.me/raw/master/fonts/HelveticaNeue-Bold.ttf

matplotlib.font_manager.fontManager.addfont('HelveticaNeue.ttf')
matplotlib.font_manager.fontManager.addfont('HelveticaNeue-Bold.ttf')

matplotlib.rc('font', family='Helvetica Neue')

# Preparing Data

In [ ]:
#Open csv file.

clinical_data = pd.read_csv("/content/drive/MyDrive/BraTS-MEN/Clinical_Data.csv", index_col='BraTS ID')
radiomic_features = pd.read_csv("/content/drive/MyDrive/BraTS-MEN/radiomic_features_train.csv", index_col=0)

data = radiomic_features.join([clinical_data], how='inner')
print(data.shape)

In [ ]:
# Filter for BraTS IDs ending with -000.

data = data[data.index.str.endswith('-000')]

print(data.shape)

In [ ]:
#Define outcome of interest.

data.loc[data['Grade'] == 1, 'OUTCOME'] = 0
data.loc[data['Grade'] == 2, 'OUTCOME'] = 1
data.loc[data['Grade'] == 3, 'OUTCOME'] = 1
data = data.dropna(subset=['OUTCOME'])

data = data.drop(list(clinical_data.columns), axis = 1)

print(data['OUTCOME'].value_counts(normalize=False, dropna=False))

In [ ]:
#Define predictor variables (x) and outcome of interest (y).

outcomes = ['OUTCOME']

x = data.drop(outcomes, axis = 1)
y = data['OUTCOME']

In [ ]:
#Check data shapes.

print(y.shape)
print(x.shape)

In [ ]:
#Split data into initial train set and test set in 80:20 ratio.

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

#Describe initial train set and test set.

print("Number patients x_train dataset:", x_train.shape[0])
print("Number patients y_train dataset:", y_train.shape[0], "\n")
print("Number patients x_test dataset:", x_test.shape[0])
print("Number patients y_test dataset:", y_test.shape[0])

In [ ]:
#Split initial train set into final train set and validation set in 75:25 ratio.

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.25, random_state = 0)

#Describe train and validation sets.

print("Number patients train_x dataset:", x_train.shape[0])
print("Number patients train_y dataset:", y_train.shape[0], "\n")
print("Number patients valid_x dataset:", x_val.shape[0])
print("Number patients valid_y dataset:", y_val.shape[0])

In [ ]:
#Apply SMOTE.

print("Before resampling, counts of label '1': {}".format(sum(y_train == 1)))
print("Before resampling, counts of label '0': {}".format(sum(y_train == 0)), "\n")

resampler = SMOTE(random_state=31)
x_train, y_train = resampler.fit_resample(x_train, y_train)

print("After resampling, counts of label '1': {}".format(sum(y_train == 1)))
print("After resampling, counts of label '0': {}".format(sum(y_train == 0)))

In [ ]:
# Scale features.

scaler = MinMaxScaler()

def scale_features(data, scaler):
    scaled_data = scaler.transform(data)
    return pd.DataFrame(scaled_data, columns=data.columns, index=data.index)

# Fit and transform the training data.
x_train = scale_features(x_train, scaler.fit(x_train))

# Transform the validation and test data.
x_val = scale_features(x_val, scaler)
x_test = scale_features(x_test, scaler)

# Feature Selection

In [ ]:
# Get a feature importance dataframe with LASSO.

def select_features_using_lasso(x, y, alpha=0.001):
    lasso = Lasso(alpha=alpha)
    lasso.fit(x, y)

    feature_importance = pd.DataFrame({
        'Feature': x.columns,
        'Importance': lasso.coef_
    })

    feature_importance = feature_importance[feature_importance['Importance'] != 0]
    feature_importance['Absolute Importance'] = feature_importance['Importance'].abs()
    feature_importance = feature_importance.set_index('Feature')
    feature_importance = feature_importance.sort_values(by='Absolute Importance', ascending=False)

    return feature_importance

feature_importance_df = select_features_using_lasso(x_train, y_train)
print("Number of selected features: ", len(feature_importance_df))

# Model Training

## TabPFN

In [ ]:
# Hyperparameter optimization for TabPFN model.

def objective_tabpfn(trial):

    try:

        N_ensemble_configurations = trial.suggest_int("N_ensemble_configurations", 2, 64, log=True)
        n_features = trial.suggest_int('n_features', 20, 100, step=1)

        selected_features = feature_importance_df.head(n_features).index.tolist()

        x_train_selected = x_train[selected_features]
        x_val_selected = x_val[selected_features]
        x_test_selected = x_test[selected_features]

        clf = TabPFNClassifier(device='cpu', N_ensemble_configurations=N_ensemble_configurations).fit(x_train_selected, y_train)

        val_preds = clf.predict(x_val_selected)

        val_auc = roc_auc_score(y_val, val_preds)

        return val_auc

    except Exception as e:

        print("Trial failed with exception")

        return None

# Create an Optuna study.
study_tabpfn = optuna.create_study(direction='maximize')

# Optimize the hyperparameters.
study_tabpfn.optimize(objective_tabpfn, n_trials=100, catch=(Exception,))

# Print the best hyperparameters and concordance index.
print("Best AUROC:", study_tabpfn.best_value, '\n')
print("Best hyperparameters:", study_tabpfn.best_params)

# Store best hyperparameters in tabpfn_param_dict.
tabpfn_param_dict = study_tabpfn.best_params

In [ ]:
# Apply feature selection for TabPFN model.

selected_features = feature_importance_df.head(tabpfn_param_dict.pop('n_features')).index.tolist()

x_train_tabpfn = x_train[selected_features]
x_val_tabpfn = x_val[selected_features]
x_test_tabpfn = x_test[selected_features]

print('x_train shape:', x_train_tabpfn.shape)
print('x_val shape:', x_val_tabpfn.shape)
print('x_test shape:', x_test_tabpfn.shape, '\n')

In [ ]:
# Fit TabPFN model.

tabpfn_model = TabPFNClassifier(device='cpu', **tabpfn_param_dict)

tabpfn_model.fit(x_train_tabpfn, y_train, overwrite_warning = True)

In [ ]:
# Calibrate TabPFN model using CalibratedClassifierCV.

calibrated_tabpfn_model = CalibratedClassifierCV(tabpfn_model, method='sigmoid', cv='prefit')
calibrated_tabpfn_model.fit(x_val_tabpfn, y_val)

In [ ]:
# Make predictions on the test set based on the trained TabPFN model.

probs_tabpfn = calibrated_tabpfn_model.predict_proba(x_test_tabpfn)
probs_tabpfn = probs_tabpfn[:, 1]

## XGBoost

In [ ]:
# Hyperparameter optimization for XGBoost model.

def objective_xgboost(trial):

    try:

        params = {
            "verbosity": 0,
            "objective":  trial.suggest_categorical("objective", ["binary:logistic"]),
            "eval_metric": "auc",
            "booster": trial.suggest_categorical("booster", ["gbtree"]),
            "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
            "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
            "max_depth" : trial.suggest_int("max_depth", 1, 9),
            "eta" : trial.suggest_float("eta", 1e-8, 1.0, log=True),
            "gamma" : trial.suggest_float("gamma", 1e-8, 1.0, log=True),
            "grow_policy" : trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
        }

        n_features = trial.suggest_int('n_features', 1,  len(feature_importance_df), step=1)

        selected_features = feature_importance_df.head(n_features).index.tolist()

        x_train_selected = x_train[selected_features]
        x_val_selected = x_val[selected_features]
        x_test_selected = x_test[selected_features]

        clf = XGBClassifier(seed=31, **params).fit(x_train_selected, y_train)
        val_preds = clf.predict(x_val_selected)

        val_auc = roc_auc_score(y_val, val_preds)

        return val_auc

    except Exception as e:

        print("Trial failed with exception")

        return None

# Create an Optuna study.
study_xgb = optuna.create_study(direction='maximize')

# Optimize the hyperparameters.
study_xgb.optimize(objective_xgboost, n_trials=100, catch=(Exception,))

# Print the best hyperparameters and concordance index.
print("Best AUROC:", study_xgb.best_value, '\n')
print("Best hyperparameters:", study_xgb.best_params)

# Store best hyperparameters in xgboost_param_dict.
xgb_param_dict = study_xgb.best_params

In [ ]:
# Apply feature selection for XGBoost model.

selected_features = feature_importance_df.head(xgb_param_dict.pop('n_features')).index.tolist()

x_train_xgb = x_train[selected_features]
x_val_xgb = x_val[selected_features]
x_test_xgb = x_test[selected_features]

print('x_train shape:', x_train_xgb.shape)
print('x_val shape:', x_val_xgb.shape)
print('x_test shape:', x_test_xgb.shape, '\n')

In [ ]:
# Fit XGBoost.

xgb_model = XGBClassifier(seed=31, **xgb_param_dict)

xgb_model.fit(x_train_xgb, y_train)

In [ ]:
# Calibrate TabPFN using CalibratedClassifierCV.

calibrated_xgb_model = CalibratedClassifierCV(xgb_model, method='sigmoid', cv='prefit')
calibrated_xgb_model.fit(x_val_xgb, y_val)

In [ ]:
# Make predictions on the test set based on the trained TabPFN model.

probs_xgb = calibrated_xgb_model.predict_proba(x_test_xgb)
probs_xgb = probs_xgb[:, 1]

## LightGBM

In [ ]:
# Hyperparameter optimization for LightGBM model.

def objective_lgb(trial):

    try:

        params = {
            "objective":  trial.suggest_categorical("objective", ["binary"]),
            "metric": "binary_logloss",
            "verbosity": -1,
            "random_state": 31,
            "boosting_type":  trial.suggest_categorical("boosting_type", ["gbdt"]),
            "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
            "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
            "num_leaves": trial.suggest_int("num_leaves", 2, 256),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
            "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
            "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        }

        n_features = trial.suggest_int('n_features', 1, len(feature_importance_df), step=1)

        selected_features = feature_importance_df.head(n_features).index.tolist()

        x_train_selected = x_train[selected_features]
        x_val_selected = x_val[selected_features]
        x_test_selected = x_test[selected_features]

        clf = LGBMClassifier(seed=31, **params).fit(x_train_selected, y_train)

        val_preds = clf.predict(x_val_selected)

        val_auc = roc_auc_score(y_val, val_preds)

        return val_auc

    except Exception as e:

        print("Trial failed with exception")

        return None

# Create an Optuna study.
study_lgb = optuna.create_study(direction='maximize')

# Optimize the hyperparameters.
study_lgb.optimize(objective_lgb, n_trials=100, catch=(Exception,))

# Print the best hyperparameters and concordance index.
print("Best AUROC:", study_lgb.best_value, '\n')
print("Best hyperparameters:", study_lgb.best_params)

# Store best hyperparameters in lgb_param_dict.
lgb_param_dict = study_lgb.best_params

In [ ]:
# Apply feature selection for LightGBM model.

selected_features = feature_importance_df.head(lgb_param_dict.pop('n_features')).index.tolist()

x_train_lgb = x_train[selected_features]
x_val_lgb = x_val[selected_features]
x_test_lgb = x_test[selected_features]

print('x_train shape:', x_train_lgb.shape)
print('x_val shape:', x_val_lgb.shape)
print('x_test shape:', x_test_lgb.shape, '\n')

In [ ]:
# Fit LightGBM model.

lgb_model = LGBMClassifier(random_state=31, **lgb_param_dict)

lgb_model.fit(x_train_lgb, y_train)

In [ ]:
# Calibrate LightGBM model using CalibratedClassifierCV.

calibrated_lgb_model = CalibratedClassifierCV(lgb_model, method='sigmoid', cv='prefit')
calibrated_lgb_model.fit(x_val_lgb, y_val)

In [ ]:
# Make predictions on the test set based on the trained LightGBM model.

probs_lgb = calibrated_lgb_model.predict_proba(x_test_lgb)
probs_lgb = probs_lgb[:, 1]

## CatBoost

In [ ]:
# Hyperparameter optimization for CatBoost model.

def objective_cb(trial):

    try:

        params = {
            "iterations": trial.suggest_int("iterations", 100, 1000),
            "depth": trial.suggest_int("depth", 4, 10),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
            "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-8, 10.0, log=True),
            "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
            "random_strength": trial.suggest_float("random_strength", 0.0, 10.0),
            "border_count": trial.suggest_int("border_count", 1, 255),
            "scale_pos_weight": trial.suggest_float("scale_pos_weight", 0.0, 10.0),
            "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
            "leaf_estimation_iterations": trial.suggest_int("leaf_estimation_iterations", 1, 10),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.4, 1.0),
            "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
            "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
        }

        n_features = trial.suggest_int('n_features', 1, len(feature_importance_df), step=1)

        selected_features = feature_importance_df.head(n_features).index.tolist()

        x_train_selected = x_train[selected_features]
        x_val_selected = x_val[selected_features]
        x_test_selected = x_test[selected_features]

        clf = CatBoostClassifier(**params).fit(x_train_selected, y_train)

        val_preds = clf.predict(x_val_selected)

        val_auc = roc_auc_score(y_val, val_preds)

        return val_auc

    except Exception as e:

        print("Trial failed with exception")

        return None

# Create an Optuna study.
study_cb = optuna.create_study(direction='maximize')

# Optimize the hyperparameters.
study_cb.optimize(objective_cb, n_trials=100, catch=(Exception,))

# Print the best hyperparameters and concordance index.
print("Best AUROC:", study_cb.best_value, '\n')
print("Best hyperparameters:", study_cb.best_params)

# Store best hyperparameters in cb_param_dict.
cb_param_dict = study_cb.best_params

In [ ]:
# Apply feature selection for CatBoost model.

selected_features = feature_importance_df.head(cb_param_dict.pop('n_features')).index.tolist()

x_train_cb = x_train[selected_features]
x_val_cb = x_val[selected_features]
x_test_cb = x_test[selected_features]

print('x_train shape:', x_train_cb.shape)
print('x_val shape:', x_val_cb.shape)
print('x_test shape:', x_test_cb.shape, '\n')

In [ ]:
# Fit CatBoost model.

cb_model = CatBoostClassifier(random_seed=31, **cb_param_dict)

cb_model.fit(x_train_cb, y_train)

In [ ]:
# Calibrate CatBoost model using CalibratedClassifierCV.

calibrated_cb_model = CalibratedClassifierCV(cb_model, method='sigmoid', cv='prefit')
calibrated_cb_model.fit(x_val_cb, y_val)

In [ ]:
# Make predictions on the test set based on the trained CatBoost model.

probs_cb = calibrated_cb_model.predict_proba(x_test_cb)
probs_cb = probs_cb[:, 1]

## Random Forest

In [ ]:
# Hyperparameter optimization for Random Forest model.

def objective_rf(trial):

    try:

        params = {
            "criterion": trial.suggest_categorical("criterion", ["gini", "entropy"]),
            "random_state": 31,
            "max_features": trial.suggest_categorical("max_features", ["auto", "sqrt","log2", None]),
            "max_depth": trial.suggest_int("max_depth", 1, 100),
            "n_estimators": trial.suggest_int("n_estimators", 100, 2000, 100),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 4, 1),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 10, 1),
        }

        n_features = trial.suggest_int('n_features', 20, len(feature_importance_df), step=1)

        selected_features = feature_importance_df.head(n_features).index.tolist()

        x_train_selected = x_train[selected_features]
        x_val_selected = x_val[selected_features]
        x_test_selected = x_test[selected_features]

        clf = RandomForestClassifier(**params).fit(x_train_selected, y_train)

        val_preds = clf.predict(x_val_selected)

        val_auc = roc_auc_score(y_val, val_preds)

        return val_auc

    except Exception as e:

        print("Trial failed with exception")

        return None

# Create an Optuna study.
study_rf = optuna.create_study(direction='maximize')

# Optimize the hyperparameters.
study_rf.optimize(objective_rf, n_trials=100, catch=(Exception,))

# Print the best hyperparameters and concordance index.
print("Best AUROC:", study_rf.best_value, '\n')
print("Best hyperparameters:", study_rf.best_params)

# Store best hyperparameters in rf_param_dict.
rf_param_dict = study_rf.best_params

In [ ]:
# Apply feature selection for Random Forest model.

selected_features = feature_importance_df.head(rf_param_dict.pop('n_features')).index.tolist()

x_train_rf = x_train[selected_features]
x_val_rf = x_val[selected_features]
x_test_rf = x_test[selected_features]

print('x_train shape:', x_train_rf.shape)
print('x_val shape:', x_val_rf.shape)
print('x_test shape:', x_test_rf.shape, '\n')

In [ ]:
# Fit Random Forest model.

rf_model = RandomForestClassifier(**rf_param_dict)

rf_model.fit(x_train_rf, y_train)

In [ ]:
# Calibrate Random Forest model using CalibratedClassifierCV.

calibrated_rf_model = CalibratedClassifierCV(rf_model, method='sigmoid', cv='prefit')
calibrated_rf_model.fit(x_val_rf, y_val)

In [ ]:
# Make predictions on the test set based on the trained Random Forest model.

probs_rf = calibrated_rf_model.predict_proba(x_test_rf)
probs_rf = probs_rf[:, 1]

# Performance Evaluation

## Metrics

In [ ]:
# Define performance evaluation function.

def evaluate_performance(y_true, y_pred_prob, confidence_level=0.95):

    #Find the optimum classification threshold.

    fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)

    optimal_idx = np.argmin(np.sqrt(np.square(1-tpr) + np.square(fpr)))
    optimal_threshold = thresholds[optimal_idx]

    y_pred = np.where(y_pred_prob >= optimal_threshold, 1, 0)

    # Calculate performance metrics.

    precision, precision_ci = bootstrap_ci(y_true=y_true, y_pred=y_pred, metric=precision_score, confidence_level=confidence_level, n_resamples=1000, method='bootstrap_bca', random_state=31)
    recall, recall_ci = bootstrap_ci(y_true=y_true, y_pred=y_pred, metric=recall_score, confidence_level=confidence_level, n_resamples=1000, method='bootstrap_bca', random_state=31)
    f1, fi_ci = bootstrap_ci(y_true=y_true, y_pred=y_pred, metric=f1_score, confidence_level=confidence_level, n_resamples=1000, method='bootstrap_bca', random_state=31)
    accuracy, accuracy_ci = bootstrap_ci(y_true=y_true, y_pred=y_pred, metric=accuracy_score, confidence_level=confidence_level, n_resamples=1000, method='bootstrap_bca', random_state=31)
    mcc, mcc_ci = bootstrap_ci(y_true=y_true, y_pred=y_pred, metric=matthews_corrcoef, confidence_level=confidence_level, n_resamples=1000, method='bootstrap_bca', random_state=31)
    auroc, auroc_ci = bootstrap_ci(y_true=y_true, y_pred=y_pred_prob, metric=roc_auc_score, confidence_level=confidence_level, n_resamples=1000, method='bootstrap_bca', random_state=31)
    auprc, auprc_ci = bootstrap_ci(y_true=y_true, y_pred=y_pred_prob, metric=average_precision_score, confidence_level=confidence_level, n_resamples=1000, method='bootstrap_bca', random_state=31)
    brier, brier_ci = bootstrap_ci(y_true=y_true, y_pred=y_pred_prob, metric=brier_score_loss, confidence_level=confidence_level, n_resamples=1000, method='bootstrap_bca', random_state=31)

    precision_ci_lower, precision_ci_upper = precision_ci
    recall_ci_lower, recall_ci_upper = recall_ci
    f1_ci_lower, f1_ci_upper = fi_ci
    accuracy_ci_lower, accuracy_ci_upper = accuracy_ci
    mcc_ci_lower, mcc_ci_upper = mcc_ci
    auroc_ci_lower, auroc_ci_upper = auroc_ci
    auprc_ci_lower, auprc_ci_upper = auprc_ci
    brier_ci_lower, brier_ci_upper = brier_ci

    precision_str = f"{precision:.3f} ({precision_ci_lower:.3f}-{precision_ci_upper:.3f})"
    recall_str = f"{recall:.3f} ({recall_ci_lower:.3f}-{recall_ci_upper:.3f})"
    f1_str = f"{f1:.3f} ({f1_ci_lower:.3f}-{f1_ci_upper:.3f})"
    accuracy_str = f"{accuracy:.3f} ({accuracy_ci_lower:.3f}-{accuracy_ci_upper:.3f})"
    mcc_str = f"{mcc:.3f} ({mcc_ci_lower:.3f}-{mcc_ci_upper:.3f})"
    auroc_str = f"{auroc:.3f} ({auroc_ci_lower:.3f}-{auroc_ci_upper:.3f})"
    auprc_str = f"{auprc:.3f} ({auprc_ci_lower:.3f}-{auprc_ci_upper:.3f})"
    brier_str = f"{brier:.3f} ({brier_ci_lower:.3f}-{brier_ci_upper:.3f})"
    optimal_threshold_str = f"{optimal_threshold*100:.1f}%"

    # Print the performance metrics.

    print("Precision:", precision_str)
    print("Recall:", recall_str)
    print("F1 Score:", f1_str)
    print("Accuracy:", accuracy_str)
    print("MCC:", mcc_str)
    print("AUROC:", auroc_str)
    print("AUPRC:", auprc_str)
    print("Brier Score:", brier_str)
    print("Optimal Classification Threshold:", optimal_threshold_str, '\n')

    metrics = {'Precision': precision_str, 'Recall': recall_str, 'F1 Score': f1_str, 'Accuracy': accuracy_str, 'MCC': mcc_str, 'AUROC': auroc_str, 'AUPRC': auprc_str, 'Brier Score': brier_str, 'Optimal Classification Threshold': optimal_threshold_str}

    return metrics

In [ ]:
# Evaluate model performances on test set.

metrics_tabpfn = evaluate_performance(y_test, probs_tabpfn, confidence_level=0.95)
metrics_xgb = evaluate_performance(y_test, probs_xgb, confidence_level=0.95)
metrics_lgb = evaluate_performance(y_test, probs_lgb, confidence_level=0.95)
metrics_cb = evaluate_performance(y_test, probs_cb, confidence_level=0.95)
metrics_rf = evaluate_performance(y_test, probs_rf, confidence_level=0.95)

results = pd.DataFrame([metrics_tabpfn, metrics_xgb, metrics_lgb, metrics_cb, metrics_rf])
results.index = ['TabPFN', 'XGBoost', 'LightGBM', 'CatBoost', 'Random Forest']
results = results.T
results.to_csv('/content/drive/MyDrive/Meningioma-Radiomics-Grading/metrics_test.csv')

## ROC Curves

In [ ]:
# Plot ROC curves for test set.

f = plt.figure()
f.set_figwidth(10)
f.set_figheight(10)

tabpfn_label = 'TabPFN Model Mean AUROC: ' + metrics_tabpfn['AUROC'].split(' ')[0]
xgb_label = 'XGBoost Model Mean AUROC: ' + metrics_xgb['AUROC'].split(' ')[0]
lgb_label = 'LightGBM Model Mean AUROC: ' + metrics_lgb['AUROC'].split(' ')[0]
cb_label = 'CatBoost Model Mean AUROC: ' + metrics_cb['AUROC'].split(' ')[0]
rf_label = 'Random Forest Model Mean AUROC: ' + metrics_rf['AUROC'].split(' ')[0]

test_fpr_tabpfn, test_tpr_tabpfn, _ = roc_curve(y_test, probs_tabpfn)
test_fpr_xgb, test_tpr_xgb, _ = roc_curve(y_test, probs_xgb)
test_fpr_lgb, test_tpr_lgb, _ = roc_curve(y_test, probs_lgb)
test_fpr_cb, test_tpr_cb, _ = roc_curve(y_test, probs_cb)
test_fpr_rf, test_tpr_rf, _ = roc_curve(y_test, probs_rf)

plt.plot(test_fpr_tabpfn, test_tpr_tabpfn, label=tabpfn_label, color='#911eb4', linewidth=3, alpha=0.8)
plt.plot(test_fpr_xgb, test_tpr_xgb, label=xgb_label, color='#4363d8', linewidth=3, alpha=0.8)
plt.plot(test_fpr_lgb, test_tpr_lgb, label=lgb_label, color='#3cb44b', linewidth=3, alpha=0.8)
plt.plot(test_fpr_cb, test_tpr_cb, label=cb_label, color='#ffe119', linewidth=3, alpha=0.8)
plt.plot(test_fpr_rf, test_tpr_rf, label=rf_label, color='#e6194B', linewidth=3, alpha=0.8)

plt.plot([0, 1], [0, 1], color='black', linestyle='--', linewidth=2, alpha=0.25, label='Random Classifier Benchmark')

plt.grid(True, which='both', linestyle='--', linewidth=0.25, color='gray', alpha=0.5)
plt.grid(True, which='minor', linestyle=':', linewidth=0.25, color='gray', alpha=0.5)

plt.gca().xaxis.set_minor_locator(MultipleLocator(0.1))
plt.gca().yaxis.set_minor_locator(MultipleLocator(0.1))

plt.ylim(-0.01, 1.01)
plt.xlim(-0.01, 1.01)

plt.tick_params(axis="y", direction="out", labelsize=18)
plt.tick_params(axis="x", direction="out", labelsize=18)

plt.text(-0.25, 0.95, 'A', fontsize=75)

plt.xlabel('False Positive Rate', fontsize=26, labelpad=12, fontweight='bold')
plt.ylabel('True Positive Rate', fontsize=26, labelpad=12, fontweight='bold')

plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), fancybox=True, shadow=False, fontsize=18)

plt.savefig('/content/drive/MyDrive/Meningioma-Radiomics-Grading/roc_test.jpg', dpi=300, bbox_inches='tight')
plt.show()

## PR Curves

In [ ]:
# Plot PR curves for test set.

f = plt.figure()
f.set_figwidth(10)
f.set_figheight(10)

tabpfn_label = 'TabPFN Model Mean AUPRC: ' + metrics_tabpfn['AUPRC'].split(' ')[0]
xgb_label = 'XGBoost Model Mean AUPRC: ' + metrics_xgb['AUPRC'].split(' ')[0]
lgb_label = 'LightGBM Model Mean AUPRC: ' + metrics_lgb['AUPRC'].split(' ')[0]
cb_label = 'CatBoost Model Mean AUPRC: ' + metrics_cb['AUPRC'].split(' ')[0]
rf_label = 'Random Forest Model Mean AUPRC: ' + metrics_rf['AUPRC'].split(' ')[0]

test_precision_tabpfn, test_recall_tabpfn, _ = precision_recall_curve(y_test, probs_tabpfn)
test_precision_xgb, test_recall_xgb, _ = precision_recall_curve(y_test, probs_xgb)
test_precision_lgb, test_recall_lgb, _ = precision_recall_curve(y_test, probs_lgb)
test_precision_cb, test_recall_cb, _ = precision_recall_curve(y_test, probs_cb)
test_precision_rf, test_recall_rf, _ = precision_recall_curve(y_test, probs_rf)

plt.plot(test_recall_tabpfn, test_precision_tabpfn, label=tabpfn_label, color='#911eb4', linewidth=3, alpha=0.8)
plt.plot(test_recall_xgb, test_precision_xgb, label=xgb_label, color='#4363d8', linewidth=3, alpha=0.8)
plt.plot(test_recall_lgb, test_precision_lgb, label=lgb_label, color='#3cb44b', linewidth=3, alpha=0.8)
plt.plot(test_recall_cb, test_precision_cb, label=cb_label, color='#ffe119', linewidth=3, alpha=0.8)
plt.plot(test_recall_rf, test_precision_rf, label=rf_label, color='#e6194B', linewidth=3, alpha=0.8)

plt.axhline(y=y_test.value_counts(normalize=True)[1.0], color='black', linestyle='--', linewidth=2, alpha = 0.25, label='Random Classifier Benchmark')

plt.grid(True, which='both', linestyle='--', linewidth=0.25, color='gray', alpha=0.5)
plt.grid(True, which='minor', linestyle=':', linewidth=0.25, color='gray', alpha=0.5)

plt.gca().xaxis.set_minor_locator(MultipleLocator(0.1))
plt.gca().yaxis.set_minor_locator(MultipleLocator(0.1))

plt.ylim(-0.01, 1.01)
plt.xlim(-0.01, 1.01)

plt.tick_params(axis="y", direction="out", labelsize=18)
plt.tick_params(axis="x", direction="out", labelsize=18)

plt.text(-0.25, 0.95, 'B', fontsize=75)

plt.xlabel('Recall', fontsize=26, labelpad=12, fontweight='bold')
plt.ylabel('Precision', fontsize=26, labelpad=12, fontweight='bold')

plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), fancybox=True, shadow=False, fontsize=18)

plt.savefig('/content/drive/MyDrive/Meningioma-Radiomics-Grading/prc_test.jpg', dpi=300, bbox_inches='tight')
plt.show()

## Calibration Curves

In [ ]:
from sklearn.calibration import calibration_curve

# Plot calibration curves for test set.

f = plt.figure()
f.set_figwidth(10)
f.set_figheight(10)

tabpfn_label = 'TabPFN Model Mean Brier Score: ' + metrics_tabpfn['Brier Score'].split(' ')[0]
xgb_label = 'XGBoost Model Mean Brier Score: ' + metrics_xgb['Brier Score'].split(' ')[0]
lgb_label = 'LightGBM Model Mean Brier Score: ' + metrics_lgb['Brier Score'].split(' ')[0]
cb_label = 'CatBoost Model Mean Brier Score: ' + metrics_cb['Brier Score'].split(' ')[0]
rf_label = 'Random Forest Model Mean Brier Score: ' + metrics_rf['Brier Score'].split(' ')[0]

x_cal_tabpfn, y_cal_tabpfn = calibration_curve(y_test, probs_tabpfn, n_bins = 6)
x_cal_xgb, y_cal_xgb = calibration_curve(y_test, probs_xgb, n_bins = 4)
x_cal_lgb, y_cal_lgb = calibration_curve(y_test, probs_lgb, n_bins = 6)
x_cal_cb, y_cal_cb = calibration_curve(y_test, probs_cb, n_bins = 6)
x_cal_rf, y_cal_rf = calibration_curve(y_test, probs_rf, n_bins = 5)

plt.plot(y_cal_tabpfn, x_cal_tabpfn, label=tabpfn_label, color='#911eb4', linewidth=3, alpha=0.8)
plt.plot(y_cal_xgb, x_cal_xgb, label=xgb_label, color='#4363d8', linewidth=3, alpha=0.8)
plt.plot(y_cal_lgb, x_cal_lgb, label=lgb_label, color='#3cb44b', linewidth=3, alpha=0.8)
plt.plot(y_cal_cb, x_cal_cb, label=cb_label, color='#ffe119', linewidth=3, alpha=0.8)
plt.plot(y_cal_rf, x_cal_rf, label=rf_label, color='#e6194B', linewidth=3, alpha=0.8)

plt.plot([0, 1], [0, 1], color='black', linestyle='--', linewidth=2, alpha=0.25, label='Ideal Calibration')

plt.grid(True, which='both', linestyle='--', linewidth=0.25, color='gray', alpha=0.5)
plt.grid(True, which='minor', linestyle=':', linewidth=0.25, color='gray', alpha=0.5)

plt.gca().xaxis.set_minor_locator(MultipleLocator(0.1))
plt.gca().yaxis.set_minor_locator(MultipleLocator(0.1))

plt.ylim(-0.01, 1.01)
plt.xlim(-0.01, 1.01)

plt.tick_params(axis="y", direction="out", labelsize=18)
plt.tick_params(axis="x", direction="out", labelsize=18)

plt.text(-0.25, 0.95, 'C', fontsize=75)

plt.xlabel('Average Predicted Probability', fontsize=26, labelpad=12, fontweight='bold')
plt.ylabel('Ratio of Positives', fontsize=26, labelpad=12, fontweight='bold')

plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), fancybox=True, shadow=False, fontsize=18)

plt.savefig('/content/drive/MyDrive/Meningioma-Radiomics-Grading/cal_test.jpg', dpi=300, bbox_inches='tight')
plt.show()

## Confusion Matrices

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

# Create the custom colormap.

cmap_tabpfn = LinearSegmentedColormap.from_list('custom_cmap', ['#FFFFFF', '#911eb4'])
cmap_xgb = LinearSegmentedColormap.from_list('custom_cmap', ['#FFFFFF', '#4363d8'])
cmap_lgb = LinearSegmentedColormap.from_list('custom_cmap', ['#FFFFFF', '#3cb44b'])
cmap_cb = LinearSegmentedColormap.from_list('custom_cmap', ['#FFFFFF', '#ffe119'])
cmap_rf = LinearSegmentedColormap.from_list('custom_cmap', ['#FFFFFF', '#e6194B'])

# Define a function for binarizing predictions.

def binarize_predictions(y_pred, threshold):
    return [1 if x >= threshold else 0 for x in y_pred]

In [ ]:
# Plot the confusion matrix for TabPFN model.

f = plt.figure()
f.set_figwidth(5)
f.set_figheight(5)

y_pred_binarized_tabpfn = binarize_predictions(probs_tabpfn, float(metrics_tabpfn['Optimal Classification Threshold'].split('%')[0]) / 100)

sns.heatmap(confusion_matrix(y_test, y_pred_binarized_tabpfn), fmt='d', annot=True, cmap=cmap_tabpfn, cbar=False,
            annot_kws={"size": 16}, linewidths=1, linecolor='black')

labels = ['Low-Grade', 'High-Grade']
plt.xticks([0.5, 1.5], labels, fontsize=16, fontweight='heavy')
plt.yticks([0.5, 1.5], labels, fontsize=16, fontweight='heavy', rotation=90)

plt.xlabel('Predicted', fontsize=18, fontweight='heavy', labelpad=16)
plt.ylabel('Truth', fontsize=18, fontweight='heavy', labelpad=16)

plt.title('A', x=-0.3, y=0.85, fontsize=65, pad=10)

plt.savefig('/content/drive/MyDrive/Meningioma-Radiomics-Grading/cm_tabpfn.jpg', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Plot the confusion matrix for TabPFN model.

f = plt.figure()
f.set_figwidth(5)
f.set_figheight(5)

y_pred_binarized_xgb = binarize_predictions(probs_xgb, float(metrics_xgb['Optimal Classification Threshold'].split('%')[0]) / 100)

sns.heatmap(confusion_matrix(y_test, y_pred_binarized_xgb), fmt='d', annot=True, cmap=cmap_xgb, cbar=False,
            annot_kws={"size": 16}, linewidths=1, linecolor='black')

labels = ['Low-Grade', 'High-Grade']
plt.xticks([0.5, 1.5], labels, fontsize=16, fontweight='heavy')
plt.yticks([0.5, 1.5], labels, fontsize=16, fontweight='heavy', rotation=90)

plt.xlabel('Predicted', fontsize=18, fontweight='heavy', labelpad=16)
plt.ylabel('Truth', fontsize=18, fontweight='heavy', labelpad=16)

plt.title('B', x=-0.3, y=0.85, fontsize=65, pad=10)

plt.savefig('/content/drive/MyDrive/Meningioma-Radiomics-Grading/cm_xgb.jpg', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Plot the confusion matrix for TabPFN model.

f = plt.figure()
f.set_figwidth(5)
f.set_figheight(5)

y_pred_binarized_lgb = binarize_predictions(probs_lgb, float(metrics_lgb['Optimal Classification Threshold'].split('%')[0]) / 100)

sns.heatmap(confusion_matrix(y_test, y_pred_binarized_lgb), fmt='d', annot=True, cmap=cmap_lgb, cbar=False,
            annot_kws={"size": 16}, linewidths=1, linecolor='black')

labels = ['Low-Grade', 'High-Grade']
plt.xticks([0.5, 1.5], labels, fontsize=16, fontweight='heavy')
plt.yticks([0.5, 1.5], labels, fontsize=16, fontweight='heavy', rotation=90)

plt.xlabel('Predicted', fontsize=18, fontweight='heavy', labelpad=16)
plt.ylabel('Truth', fontsize=18, fontweight='heavy', labelpad=16)

plt.title('C', x=-0.3, y=0.85, fontsize=65, pad=10)

plt.savefig('/content/drive/MyDrive/Meningioma-Radiomics-Grading/cm_lgb.jpg', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Plot the confusion matrix for TabPFN model.

f = plt.figure()
f.set_figwidth(5)
f.set_figheight(5)

y_pred_binarized_cb = binarize_predictions(probs_cb, float(metrics_cb['Optimal Classification Threshold'].split('%')[0]) / 100)

sns.heatmap(confusion_matrix(y_test, y_pred_binarized_cb), fmt='d', annot=True, cmap=cmap_cb, cbar=False,
            annot_kws={"size": 16}, linewidths=1, linecolor='black')

labels = ['Low-Grade', 'High-Grade']
plt.xticks([0.5, 1.5], labels, fontsize=16, fontweight='heavy')
plt.yticks([0.5, 1.5], labels, fontsize=16, fontweight='heavy', rotation=90)

plt.xlabel('Predicted', fontsize=18, fontweight='heavy', labelpad=16)
plt.ylabel('Truth', fontsize=18, fontweight='heavy', labelpad=16)

plt.title('D', x=-0.3, y=0.85, fontsize=65, pad=10)

plt.savefig('/content/drive/MyDrive/Meningioma-Radiomics-Grading/cm_cb.jpg', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Plot the confusion matrix for TabPFN model.

f = plt.figure()
f.set_figwidth(5)
f.set_figheight(5)

y_pred_binarized_rf = binarize_predictions(probs_rf, float(metrics_rf['Optimal Classification Threshold'].split('%')[0]) / 100)

sns.heatmap(confusion_matrix(y_test, y_pred_binarized_rf), fmt='d', annot=True, cmap=cmap_rf, cbar=False,
            annot_kws={"size": 16}, linewidths=1, linecolor='black')

labels = ['Low-Grade', 'High-Grade']
plt.xticks([0.5, 1.5], labels, fontsize=16, fontweight='heavy')
plt.yticks([0.5, 1.5], labels, fontsize=16, fontweight='heavy', rotation=90)

plt.xlabel('Predicted', fontsize=18, fontweight='heavy', labelpad=16)
plt.ylabel('Truth', fontsize=18, fontweight='heavy', labelpad=16)

plt.title('E', x=-0.3, y=0.85, fontsize=65, pad=10)

plt.savefig('/content/drive/MyDrive/Meningioma-Radiomics-Grading/cm_rf.jpg', dpi=300, bbox_inches='tight')
plt.show()

# SHAP Plots

In [ ]:
#Calculate SHAP values for TabPFN.

tabpfn_explainer = shap.Explainer(calibrated_tabpfn_model.predict, x_test_tabpfn)
tabpfn_shap_values = tabpfn_explainer(x_test_tabpfn)

In [ ]:
# Plot SHAP bar plot for TabPFN.

shap.plots.bar(tabpfn_shap_values, max_display=20, show=False)

fig = plt.gcf()
ax = plt.gca()
fig.set_figheight(15)
fig.set_figwidth(5)

# Change the color of the bars
for bar in ax.patches:
    bar.set_color('#911eb4')  # Change this to your desired color

# Change the color of the numbers next to the bars
for text in ax.texts:
    text.set_color('black')

#plt.title('B', x=-1, y=1, fontsize=50, pad=20)
plt.xlabel("Mean |SHAP Value|", fontsize=14, fontweight='heavy', labelpad=8)
plt.tick_params(axis="y", direction="out", labelsize=14)
plt.tick_params(axis="x", direction="out", labelsize=14)

plt.savefig('/content/drive/MyDrive/Meningioma-Radiomics-Grading/shap_tabpfn_test.jpg', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#Calculate SHAP values for XGBoost.

xgb_explainer = shap.Explainer(calibrated_xgb_model.predict, x_test_xgb)
xgb_shap_values = xgb_explainer(x_test_xgb)

In [ ]:
# Plot SHAP bar plot for XGBoost

shap.plots.bar(xgb_shap_values, max_display=20, show=False)

fig = plt.gcf()
ax = plt.gca()
fig.set_figheight(15)
fig.set_figwidth(5)

# Change the color of the bars
for bar in ax.patches:
    bar.set_color('#1f77b4')  # Change this to your desired color

# Change the color of the numbers next to the bars
for text in ax.texts:
    text.set_color('black')

#plt.title('B', x=-1, y=1, fontsize=50, pad=20)
plt.xlabel("Mean |SHAP Value|", fontsize=14, fontweight='heavy', labelpad=8)
plt.tick_params(axis="y", direction="out", labelsize=14)
plt.tick_params(axis="x", direction="out", labelsize=14)

plt.savefig('/content/drive/MyDrive/Meningioma-Radiomics-Grading/shap_xgb_test.jpg', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#Calculate SHAP values for LightGBM.

lgb_explainer = shap.Explainer(calibrated_lgb_model.predict, x_test_lgb)
lgb_shap_values = lgb_explainer(x_test_lgb)

In [ ]:
# Plot SHAP bar plot for LightGBM.

shap.plots.bar(lgb_shap_values, max_display=20, show=False)

fig = plt.gcf()
ax = plt.gca()
fig.set_figheight(15)
fig.set_figwidth(5)

# Change the color of the bars
for bar in ax.patches:
    bar.set_color('#3cb44b')  # Change this to your desired color

# Change the color of the numbers next to the bars
for text in ax.texts:
    text.set_color('black')

#plt.title('B', x=-1, y=1, fontsize=50, pad=20)
plt.xlabel("Mean |SHAP Value|", fontsize=14, fontweight='heavy', labelpad=8)
plt.tick_params(axis="y", direction="out", labelsize=14)
plt.tick_params(axis="x", direction="out", labelsize=14)

plt.savefig('/content/drive/MyDrive/Meningioma-Radiomics-Grading/shap_lgb_test.jpg', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#Calculate SHAP values for CatBoost.

cb_explainer = shap.Explainer(calibrated_cb_model.predict, x_test_cb)
cb_shap_values = cb_explainer(x_test_cb)

In [ ]:
# Plot SHAP bar plot for CatBoost.

shap.plots.bar(cb_shap_values, max_display=20, show=False)

fig = plt.gcf()
ax = plt.gca()
fig.set_figheight(15)
fig.set_figwidth(5)

# Change the color of the bars
for bar in ax.patches:
    bar.set_color('#ffe119')  # Change this to your desired color

# Change the color of the numbers next to the bars
for text in ax.texts:
    text.set_color('black')

#plt.title('B', x=-1, y=1, fontsize=50, pad=20)
plt.xlabel("Mean |SHAP Value|", fontsize=14, fontweight='heavy', labelpad=8)
plt.tick_params(axis="y", direction="out", labelsize=14)
plt.tick_params(axis="x", direction="out", labelsize=14)

plt.savefig('/content/drive/MyDrive/Meningioma-Radiomics-Grading/shap_cb_test.jpg', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#Calculate SHAP values for Random Forest.

rf_explainer = shap.Explainer(calibrated_rf_model.predict, x_test_rf)
rf_shap_values = rf_explainer(x_test_rf)

In [ ]:
# Plot SHAP bar plot for Random Forest.

shap.plots.bar(rf_shap_values, max_display=20, show=False)

fig = plt.gcf()
ax = plt.gca()
fig.set_figheight(15)
fig.set_figwidth(5)

# Change the color of the bars
for bar in ax.patches:
    bar.set_color('#e6194B')  # Change this to your desired color

# Change the color of the numbers next to the bars
for text in ax.texts:
    text.set_color('black')

#plt.title('B', x=-1, y=1, fontsize=50, pad=20)
plt.xlabel("Mean |SHAP Value|", fontsize=14, fontweight='heavy', labelpad=8)
plt.tick_params(axis="y", direction="out", labelsize=14)
plt.tick_params(axis="x", direction="out", labelsize=14)

plt.savefig('/content/drive/MyDrive/Meningioma-Radiomics-Grading/shap_rf_test.jpg', dpi=300, bbox_inches='tight')
plt.show()